In [17]:
import os
import numpy as np
import torch

device = 'cpu'
block_size = 1023
batch_size = 2
dataset = ''

# poor man's data loader
data_dir = dataset
train_data = np.memmap(os.path.join(data_dir, 'train_elocondition.bin'), dtype=np.uint8, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val_elocondition.bin'), dtype=np.uint8, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # Ensure the starting index is a multiple of block_size
    ix = torch.randint(0, len(data) // (block_size + 1), (batch_size,)) * (block_size + 1)
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [18]:
print(len(train_data))
print(len(val_data) / 1024)

14642907136
144442.0


In [19]:
import pickle
import os


# Usage
# meta_file_path = os.path.join(os.path.dirname(__file__), 'meta.pkl')
file_path = 'meta.pkl'
with open(file_path, 'rb') as f:
    data = pickle.load(f)
print(data)

itos = data['itos']

def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

{'vocab_size': 32, 'itos': {0: ' ', 1: '#', 2: '+', 3: '-', 4: '.', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: ';', 16: '=', 17: 'B', 18: 'K', 19: 'N', 20: 'O', 21: 'Q', 22: 'R', 23: 'a', 24: 'b', 25: 'c', 26: 'd', 27: 'e', 28: 'f', 29: 'g', 30: 'h', 31: 'x'}, 'stoi': {' ': 0, '#': 1, '+': 2, '-': 3, '.': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, ';': 15, '=': 16, 'B': 17, 'K': 18, 'N': 19, 'O': 20, 'Q': 21, 'R': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'x': 31}}


In [20]:
X, Y = get_batch('val')
print("Batch")
print(X)
print("y")
print(Y)

Batch
tensor([[15,  6, 12,  ...,  4, 28, 31],
        [15,  6, 11,  ..., 11,  0, 18]])
y
tensor([[ 6, 12, 14,  ..., 28, 31, 29],
        [ 6, 11,  5,  ...,  0, 18, 24]])


In [21]:
print(decode(X[0].tolist()))
print(decode(X[1].tolist()))
print(decode(Y[0].tolist()))
print(decode(Y[1].tolist()))

;1799 1806 1.d4 d5 2.c4 c6 3.Nf3 Nf6 4.Nc3 Bf5 5.cxd5 cxd5 6.Qb3 Qb6 7.Nxd5 Qxb3 8.Nxf6+ gxf6 9.axb3 Nc6 10.e3 e5 11.d5 Bb4+ 12.Bd2 Bxd2+ 13.Nxd2 Nb4 14.Bb5+ Kf8 15.O-O Nxd5 16.e4 Nc7 17.Bc4 Be6 18.Rfd1 Ke7 19.Nf1 Rhd8 20.Ng3 b5 21.Bxe6 fxe6 22.Rdc1 Rd7 23.Rc6 a6 24.Rac1 Ra7 25.Nh5 Ne8 26.Rc8 Nd6 27.Rh8 Nxe4 28.Rxh7+ Kd8 29.Rh8+ Ke7 30.Rh7+ Kd6 31.Rd1+ Kc6 32.Rc1+ Kb6 33.Rh8 Rd2 34.f3 Nf2 35.b4 Nd3 36.Rc3 Rd1#;1387 2095 1.e3 d5 2.Be2 e6 3.g4 Nf6 4.f4 Nfd7 5.g5 c5 6.Kf2 h6 7.Nf3 hxg5 8.Nxg5 Nf6 9.Nf3 Ne4+ 10.Kf1 Be7 11.d3 Bh4 12.dxe4 dxe4 13.Ng5 Bxg5 14.fxg5 Qxg5 15.Rg1 Qf6+ 16.Kg2 Rxh2+ 17.Kxh2 Qf2+ 18.Rg2 Qh4+ 19.Kg1 Qh1+ 20.Kxh1 Nc6 21.Kg1 Bd7 22.Rg6 Kd8 23.Rxg7 Kc7 24.Rxf7 Rf8 25.Rxd7+ Kb6 26.Bg4 Rg8;1698 1686 1.e4 c5 2.Nc3 e6 3.Bc4 Nc6 4.Nf3 Nf6 5.d3 g6 6.Bg5 Bg7 7.e5 Qb6 8.exf6 Bf8 9.b3 Na5 10.Rb1 d5 11.b4 cxb4 12.Na4 Qc6 13.Bb3 Nxb3 14.axb3 b5 15.Nb2 a5 16.Ne5 Qc3+ 17.Qd2 Qxe5+ 18.Kd1 Bc5 19.f4 Qc3 20.Qxc3 bxc3 21.Na4 bxa4 22.bxa4 Bd7 23.Ra1 Bf8 24.Re1 h5 25.Re3 h4 26.h3 Bh6 27.d